In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))
sys.path.append(os.path.join(os.getcwd(), "molgen"))

%load_ext autoreload
%autoreload 2

import torch
from slot_attention import SlotAttention

from otgnn.models import GCN
# from molgen.dataloading.MolGraphBatchPreprocessor import MolGraph
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model
from otgnn.graph import SYMBOLS, FORMAL_CHARGES, BOND_TYPES


from mol_opt2.mol_opt.data_mol_opt import MolOptDataset
from mol_opt2.mol_opt.data_mol_opt import get_loader
from mol_opt2.mol_opt.arguments import get_args
from mol_opt2.mol_opt.train_mol_opt import main
from mol_opt2.mol_opt.mol_opt import MolOpt
from mol_opt2.mol_opt.decoder_mol_opt import MolOptDecoder
from mol_opt2.mol_opt.ot_utils import encode_target
from mol_opt2.mol_opt.task_metrics import props

from molgen.dataloading.feat2smiles import feat2smiles
from molgen.dataloading.mol_drawer import MolDrawer
from molgen.metrics.Penalty import Penalty

from rdkit.Chem.Draw import IPythonConsole

/home/octav/gitrepos/tum-thesis


In [2]:
import sys
print (sys.path)

['/home/octav/anaconda3/envs/mol_ot/lib/python36.zip', '/home/octav/anaconda3/envs/mol_ot/lib/python3.6', '/home/octav/anaconda3/envs/mol_ot/lib/python3.6/lib-dynload', '', '/home/octav/anaconda3/envs/mol_ot/lib/python3.6/site-packages', '/home/octav/anaconda3/envs/mol_ot/lib/python3.6/site-packages/IPython/extensions', '/home/octav/.ipython', '/home/octav/gitrepos/tum-thesis/otgnn', '/home/octav/gitrepos/tum-thesis/molgen', '/home/octav/gitrepos/tum-thesis/otgnn', '/home/octav/gitrepos/tum-thesis/molgen', '..', '..', '/home/octav/gitrepos/tum-thesis/iclr19-graph2graph/props', '..', '..', '/home/octav/gitrepos/tum-thesis/iclr19-graph2graph/props']


In [3]:
slot_attn = SlotAttention(
    num_slots = 5,
    dim = 512,
    iters = 3   # iterations of attention, defaults to 3
)

inputs = torch.randn(2, 1024, 512)
slot_attn(inputs) # (2, 5, 512)

tensor([[[-2.3905e-01,  3.7804e-01, -5.8688e-01,  ..., -3.4940e-01,
          -4.8382e-01,  9.0381e-04],
         [-9.6186e-01,  3.4954e-01, -2.9336e-01,  ..., -4.0235e-01,
          -2.3043e-01, -1.0498e-01],
         [-1.9537e-01,  1.9760e-01, -4.3224e-01,  ..., -4.4038e-01,
           1.4511e-01, -4.7906e-02],
         [-4.4555e-01,  8.6506e-01, -6.6218e-01,  ..., -3.8130e-01,
          -4.7370e-01,  1.6384e-01],
         [-5.8587e-01,  5.1695e-02, -5.8667e-01,  ..., -8.5430e-01,
          -8.3580e-02, -4.5780e-01]],

        [[-8.9125e-01,  8.1932e-01, -5.2527e-01,  ..., -9.7158e-02,
          -1.3352e-01, -1.1089e-01],
         [-2.4210e-01,  1.9234e-01, -9.0357e-01,  ..., -1.3499e-01,
           1.1638e-01, -2.7185e-01],
         [-1.9571e-01,  2.6238e-01, -3.9523e-01,  ..., -3.2779e-01,
           1.3290e-01, -1.6547e-01],
         [-7.4695e-01,  6.0923e-01, -1.5586e+00,  ..., -4.2516e-01,
          -5.0520e-02, -2.3505e-01],
         [-3.9515e-01,  3.0549e-01, -4.1426e-01,  ...

In [4]:
slot_attn(inputs).shape

torch.Size([2, 5, 512])

In [5]:
slot_attn(inputs, num_slots = 8).shape

torch.Size([2, 8, 512])

# Try out implementation

In [6]:
sys.argv = ["", "-cuda"]
args = get_args()
args.output_dir = "mol_opt2/output/"

In [7]:
molopt = MolOpt(args).to(device = args.device)
molopt_decoder = MolOptDecoder(args).to(device = args.device)

In [8]:
train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train", 96, False)
val_data_loader = get_loader("iclr19-graph2graph/data/qed", "val", 96, False)

for i in val_data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break

In [9]:
molopt(X).shape

torch.Size([2238, 50])

In [10]:
sum([x for _, x in X.scope])

2238

In [15]:
# overall flow
x_embedding = molopt.forward(X)
yhat_logits = molopt_decoder.forward(x_embedding, X, Y)
yhat_labels = molopt_decoder.discretize(*yhat_logits)

In [17]:
molopt_decoder

MolOptDecoder(
  (slot_att): SlotAttention(
    (to_q): Linear(in_features=50, out_features=50, bias=True)
    (to_k): Linear(in_features=50, out_features=50, bias=True)
    (to_v): Linear(in_features=50, out_features=50, bias=True)
    (gru): GRUCell(50, 50)
    (mlp): Sequential(
      (0): Linear(in_features=50, out_features=128, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=128, out_features=50, bias=True)
    )
    (norm_input): LayerNorm((50,), eps=1e-05, elementwise_affine=True)
    (norm_slots): LayerNorm((50,), eps=1e-05, elementwise_affine=True)
    (norm_pre_ff): LayerNorm((50,), eps=1e-05, elementwise_affine=True)
  )
  (fc1_SYMBOLS): Linear(in_features=50, out_features=100, bias=True)
  (fc2_SYMBOLS): Linear(in_features=100, out_features=64, bias=True)
  (fc1_CHARGES): Linear(in_features=50, out_features=100, bias=True)
  (fc2_CHARGES): Linear(in_features=100, out_features=5, bias=True)
  (fc1_BONDS): Linear(in_features=100, out_features=100, bias=